## Antepost horse racing market scraper 

this workbook returns a list of horses who are running today and are in antepost future betting markets this is so odds can be adjusted accordingly reflecting the updated probabilities after they have ran today

In [23]:
#import libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [24]:
#funtion that scrapes sporting life for todays horses
def todays_runners(url): 
    today = requests.get(url)
    todays_entries = BeautifulSoup(today.content,'lxml')
    horse_name = [some.text for some in todays_entries.find_all('td', {'class':'horse-name'})]
    race = [some_more.text for some_more in todays_entries.find_all('td', {'class':'race-name'})]
    return list(zip(horse_name,race))

In [25]:
#url 
runners = todays_runners('https://www.sportinglife.com/racing/abc-guide')

In [26]:
#clean up and put into a dataframe todays details 
df = pd.DataFrame(runners, columns=['horse','race and time'])
df['str_splt'] = df['race and time'].str.split(" ")
df['race'] = df['str_splt'].str.get(1)
df['time'] = df['str_splt'].str.get(0)
df = df.drop(['race and time','str_splt'], axis = 1)

In [27]:
df.tail()

,horse,race,time
304,You Bank,NaasTue,12:30
305,Young Phoenix,FakenhamTue,13:20
306,Young Ted,NaasTue,12:30
307,Zapper Cass,SouthwellTue,13:30
308,Zeebad,SouthwellTue,14:30


In [28]:
'''define a function that gets, parses html and returns a dataframe
when given the url'''

def df_make(url):    
    
    test = requests.get(url)
    test_soup = BeautifulSoup(test.content,'lxml')
    article = test_soup.find_all('div',class_='fixture')
    
    races = []
    favs = []
    odds = []
    date = []
    vao = 'View all oddsView all odds'

    for race in article:
        header = race.find('h2',class_='fixture-date').text
        vao = 'View all oddsView all odds'
        header = header.replace(vao, '')
        races.append(header)

    for horse in article:
        fav = horse.find('span', {'class':'fixtures-bet-name beta-footnote'}).text
        favs.append(fav)  


    for odd in article:
        odd = odd.find('span', {'class':'fixtures-bet-odds beta-footnote'}).text
        odds.append(odd) 
        
    for race in article:
        header = race.find('p', class_='text full-bar beta-caption2').text
        date.append(header)
  
    
    df = pd.DataFrame()
    df['races'] = races
    df['horse'] = favs
    df['odds'] = odds
    df['date_and_time'] = date
    
    df['str_split'] = df.odds.str.split('/')

    df['num'] = df['str_split'].str.get(0).astype(float)
    df['den'] = df['str_split'].str.get(1).astype(float)

    df.den = df.den.fillna(1)
    df['dec_odds'] = (df['num']/df['den'])+1
    
    df = df.drop(['odds','str_split','num','den'], axis =1)
    
    return df 
    

In [29]:
#two urls required for passing 
antepost_runners = df_make(antepost)
cheltenham_runners = df_make(cheltenham)


In [30]:
#join one df on to the other 
all_favs = antepost_runners.append(cheltenham_runners).reset_index().drop('index',axis = 1)

to get the name of the race for all runners I need to use the favourite for each race as the starting point to index from, I can then use pandas forward fill method to attach the correct race to the other horses name until the next fav/index

In [45]:
#the favourites in all upcoming races
all_favs

,races,horse,date_and_time,dec_odds
0,Sky Bet Supreme Trial Novices Hurdle,Angels Breathout,Friday 21st December 2018,2.750000
1,My Pension Expert Handicap Chase,Janika,Saturday 22nd December 2018,7.000000
2,50 Risk Free On Racing With Smarkets Handicap ...,Always Resolute,Saturday 22nd December 2018,6.000000
3,Long Walk Hurdle,Call Me Lord,Saturday 22nd December 2018,3.000000
4,Smarkets Tommy Whittle Handicap Chase,Kimberlite Candy,Saturday 22nd December 2018,5.000000
5,Silver Cup Handicap Chase,Gold Presentin,Saturday 22nd December 2018,5.000000
6,Betfair Exchange Trophy,Jollys Cracked It,Saturday 22nd December 2018,7.500000
7,Racing Post Novice Chase,Mengli Khan,Wednesday 26th December 2018,1.800000
8,King George VI Chase,Might Bite,Wednesday 26th December 2018,4.000000
9,Paddys Rewards Club Sugar Paddy Chase,Footpad,Thursday 27th December 2018,2.200000


In [31]:
#create a left join with the favourites in the correct place
final = pd.merge(all_runners,all_favs, how ='left', left_on = ['horse','dec_odds'], right_on = ['horse','dec_odds'])

In [44]:
final.head()

,horse,dec_odds,races,date_and_time
0,Annamix,10.0,Supreme Novice Hurdle,Tuesday 12th March 2019
1,Champ,17.0,Supreme Novice Hurdle,Tuesday 12th March 2019
2,Al Dancer,21.0,Supreme Novice Hurdle,Tuesday 12th March 2019
3,Battleoverdoyen,21.0,Supreme Novice Hurdle,Tuesday 12th March 2019
4,Didtheyleaveuoutto,21.0,Supreme Novice Hurdle,Tuesday 12th March 2019


In [32]:
#forward fill the Nans so all the races and dates have correct values
final.races = final.races.fillna(method = 'ffill')
final.date_and_time = final.date_and_time.fillna(method = 'ffill')

In [33]:
#inner join the list of all runners with those running today leaving only those in both lists
todays_runners = pd.merge(df,final, how ='inner', on = 'horse')

In [34]:
# set the index to time for clarity 
todays_runners = todays_runners.set_index('time').sort_index()

In [40]:
todays_runners

,Horse,Todays Race,Decimal Odds,Future Engagements,Date and time of race
time,,,,,
12:30,Embrun Mitja,NaasTue,26.0,Ballymore Novice Hurdle,Thursday 14th March 2019
12:40,Fabianski,CatterickTue,26.0,Fred Winter Juvenile Handicap Hurdle,Wednesday 13th March 2019
13:35,Duc Des Genievres,NaasTue,26.0,National Hunt Chase,Tuesday 12th March 2019
13:35,Ex Patriot,NaasTue,41.0,National Hunt Chase,Tuesday 12th March 2019
14:05,Barrington Court,NaasTue,34.0,Champion Bumper,Wednesday 13th March 2019
14:05,Barrington Court,NaasTue,15.0,Mares Novice Hurdle,Thursday 14th March 2019
14:05,Relegate,NaasTue,41.0,Supreme Novice Hurdle,Tuesday 12th March 2019
14:05,Relegate,NaasTue,34.0,Ballymore Novice Hurdle,Thursday 14th March 2019
15:20,Doncesar De Pretot,FakenhamTue,51.0,Ballymore Novice Hurdle,Thursday 14th March 2019


In [36]:
#change column headers 
todays_runners.columns = ['Horse', 'Todays Race', 'Decimal Odds', 'Future Engagements', 'Date and time of race']

In [37]:
#save to Excel sheet to print 
todays_runners.to_excel('saturday.xlsx')